In [0]:
 import gym
 import torch
 import torch.nn as n
 import torch.nn.functional as f
 import numpy as np
 import matplotlib.pyplot as plt
 import torch.optim as op

In [0]:
!pip install ptan

     |████████████████████████████████| 773.1MB 20kB/s 
  Created wheel for ptan: filename=ptan-0.6-cp36-none-any.whl size=23502 sha256=ebc3958bfe6cba5d97db799d8d529211ed1ef48271f7c8a31fbd6c31c907e169
  Stored in directory: /root/.cache/pip/wheels/f0/4b/2f/9a45fd39b0a614a2716bc6128a7f1adb4647f323a2d90783f2
Successfully built ptan
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [0]:
import ptan

In [0]:
env=gym.make("CartPole-v0")

In [0]:
obsdim=env.observation_space.shape[0]
nactions=env.action_space.n

In [0]:
class net(n.Module):      # gives out action values given state x for each action
  def __init__(self,obsdim,nactions):
    super().__init__()
    self.obsdim=obsdim
    self.nact=nactions
    self.fc1=n.Linear(obsdim,128)
    self.fc2=n.Linear(128,nactions)
  
  def forward(self,x):
    x=x.float()
    out=f.relu(self.fc1(x))
    return self.fc2(out)

In [0]:
model=net(obsdim,nactions)

In [0]:
# experiment source for getting s a r nexts
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
eps=0.1

In [0]:
actionselector=ptan.actions.EpsilonGreedyActionSelector(epsilon=eps)
agent=ptan.agent.DQNAgent(model,actionselector,device=device)

In [0]:
expsource=ptan.experience.ExperienceSourceFirstLast(env,agent,gamma=0.99,steps_count=1)

In [0]:
count=0
for e in expsource:
  print(e.state,"--",e.last_state,"--",e.reward,"--",expsource.pop_total_rewards())
  count+=1
  if count==20:
    break

# Before appending we should check pop total rewards if it is non zero whatever states actions rewards last states in previous list can be
# used for a single episode

[0.03485613 0.02608345 0.02001394 0.04839476] -- [ 0.0353778   0.22091278  0.02098183 -0.23790701] -- 1.0 -- []
[ 0.0353778   0.22091278  0.02098183 -0.23790701] -- [ 0.03979605  0.4157288   0.01622369 -0.5238985 ] -- 1.0 -- []
[ 0.03979605  0.4157288   0.01622369 -0.5238985 ] -- [ 0.04811063  0.61061871  0.00574572 -0.81142534] -- 1.0 -- []
[ 0.04811063  0.61061871  0.00574572 -0.81142534] -- [ 0.060323    0.80566148 -0.01048278 -1.10229543] -- 1.0 -- []
[ 0.060323    0.80566148 -0.01048278 -1.10229543] -- [ 0.07643623  1.00091976 -0.03252869 -1.39824865] -- 1.0 -- []
[ 0.07643623  1.00091976 -0.03252869 -1.39824865] -- [ 0.09645463  1.19643073 -0.06049367 -1.7009217 ] -- 1.0 -- []
[ 0.09645463  1.19643073 -0.06049367 -1.7009217 ] -- [ 0.12038324  1.39219513 -0.0945121  -2.01180547] -- 1.0 -- []
[ 0.12038324  1.39219513 -0.0945121  -2.01180547] -- [ 0.14822715  1.58816412 -0.13474821 -2.33219227] -- 1.0 -- []
[ 0.14822715  1.58816412 -0.13474821 -2.33219227] -- [ 0.17999043  1.7842234

In [0]:
buffer=ptan.experience.ExperienceReplayBuffer(expsource,10000)

In [0]:
optimizer=op.Adam(model.parameters(),lr=0.001)

In [0]:
tgtmodel=ptan.agent.TargetNet(model)

In [0]:
# parameters
stopreward=500
gamma=0.99
batchsize=64

In [0]:
class rewardtracker:
  def __init__(self,stopreward):
    self.stopreward=stopreward
    self.totalreward=[]
  
  def reward(self,frame,reward1):
    self.totalreward.append(reward1)
    meanreward=np.mean(self.totalreward[-100:])
    if meanreward>stopreward:
      return True
    return False
    

In [0]:
def unpackbatch(batch):
  states,actions,rewards,dones,nextstate=[],[],[],[],[]

  for st in batch:
    states.append(np.array(st.state))
    actions.append(np.array(st.action))
    rewards.append(np.array(st.reward))
    dones.append(np.array(st.last_state is None))
    if st.last_state is None:

      nextstate.append(np.array(st.state))
    else:
      nextstate.append(np.array(st.last_state))
    return np.array(states),np.array(actions),np.array(rewards),np.array(dones),np.array(nextstate)  


In [0]:
def calcloss(batch,net,tgtnet,gamma,device):
  batchs,batchac,batchrew,batchdone,batchns=unpackbatch(batch)
  batchs=torch.tensor(batchs).to(device)
  batchac=torch.tensor(batchac).to(device)
  batchrew=torch.tensor(batchrew).to(device)
  batchdone=torch.BoolTensor(batchdone).to(device)
  batchns=torch.tensor(batchns).to(device)
  actionqval=net(batchs).gather(1,batchac.unsqueeze(-1)).squeeze(-1)
  nextstateactqval=tgtnet(batchns).max(1)[0]
  nextstateactqval[batchdone]=0.0

  return n.MSELoss()(actionqval,(batchrew+gamma*nextstateactqval))


In [0]:
rt=rewardtracker(stopreward)

In [0]:
tgtmodel.target_model

net(
  (fc1): Linear(in_features=4, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [0]:
tgtmodel.model

net(
  (fc1): Linear(in_features=4, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [0]:
device1=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
frameidx=0
rew=[]
while True:
  frameidx+=1
  buffer.populate(1)
  newrewards=expsource.pop_total_rewards()
  if newrewards:
    rew.append(newrewards[0])
  if newrewards:
    if rt.reward(frameidx,newrewards[0]):
      break
  
  if len(buffer)<5000:
    continue
  
  optimizer.zero_grad()
  batch=buffer.sample(batchsize)
  lossv=calcloss(batch,model,tgtmodel.target_model,gamma=gamma,device=device1)
  lossv.backward()
  optimizer.step()

  if frameidx%5000==0:
    print(lossv.item())
    print(np.mean(rew))
  if frameidx%1000==0:
    tgtmodel.sync()

1.0185098578367615
113.4888888888889
0.42069879273185506
125.1604938271605
0.361996779742185
67.82352941176471
0.06158593205327634
75.77735849056604
0.21776935161688016
82.16339869281046
0.1046582130366005
88.54867256637168
0.11551970388973132
94.37903225806451
0.008739065902773291
99.82835820895522
0.08628278743708506
104.76566125290023
0.055246803909540176
109.43326039387308
0.32084196858340874
114.19047619047619
0.11596095026470721
118.20275590551181
1.4040022427216172
120.37892791127541
0.10520686605013907
122.4493006993007
0.009853749128524214
122.28175895765472
349.71827770177333
108.76662143826323
5.800434482807759
109.25064267352185
0.0414583683013916
109.77344701583435
0.46495642891386524
109.78521939953811
0.06269088038243353
111.93064876957494
9.178581042448059
114.14115092290989
0.04531592153944075
116.32101372756071
0.031635629071388394
118.47325102880659
0.2787919277907349
120.51755265797392
0.24817229807376862
122.2314453125
0.6223668116726913
123.57549857549857
0.370845

KeyboardInterrupt: ignored